In [ ]:
!pip install pytorch-tabnet

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.display.max_columns = 999

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.neural_network import MLPClassifier,MLPRegressor

from sklearn.metrics import accuracy_score, mean_squared_error

from pytorch_tabnet.tab_model import TabNetClassifier

import matplotlib.pyplot as plt


In [ ]:
play_data = pd.read_csv('../input/nfl-big-data-bowl-2022/plays.csv')
player_data = pd.read_csv('../input/nfl-big-data-bowl-2022/players.csv')
pff_data = pd.read_csv('../input/nfl-big-data-bowl-2022/PFFScoutingData.csv')

In [ ]:
kickoff_play = play_data[(play_data.specialTeamsPlayType == 'Kickoff')]
kickoff_play = pd.merge(kickoff_play,pff_data)

In [ ]:
tracking_data = []

for year in range(2018,2021):
    data = pd.read_csv('../input/nfl-big-data-bowl-2022/tracking'+str(year) + '.csv')
    data = pd.merge(data,kickoff_play[['gameId','playId']])
#     data = data[data.team == 'football']
    tracking_data.append(data)
tracking_data = pd.concat(tracking_data).reset_index(drop=True)

In [ ]:
tracking_data.loc[tracking_data.playDirection == 'left','x'] = 120-tracking_data['x']
tracking_data.loc[tracking_data.playDirection == 'left','y'] = 53.33-tracking_data['y']

tracking_data['x_dir'] = -np.sin((np.pi / 180) * tracking_data.dir)
tracking_data['y_dir'] = -np.cos((np.pi / 180) * tracking_data.dir)
tracking_data['x_s'] = tracking_data.x_dir * tracking_data.s
tracking_data['y_s'] = tracking_data.y_dir * tracking_data.s

In [ ]:
kickoff_frame = tracking_data[(tracking_data.event == 'kickoff')&(tracking_data.displayName == 'football')].groupby(['gameId','playId'])[['frameId','x','y']].first().reset_index()
kickoff_frame = pd.merge(kickoff_frame, kickoff_play[['gameId','playId','hangTime','kickContactType','kickType','kickReturnYardage','playDescription','specialTeamsResult','kickerId','returnerId','penaltyYards']])
kickoff_frame = kickoff_frame.dropna(subset=['hangTime'])

In [ ]:
tracking_data = pd.merge(tracking_data,kickoff_frame[['playId','gameId','frameId','hangTime','x','y','kickContactType','kickType','kickReturnYardage','playDescription','specialTeamsResult','kickerId','returnerId','penaltyYards']],on=['playId','gameId'],suffixes=['','_kickoff'])
tracking_data['ground_frame'] = tracking_data['frameId_kickoff']+(tracking_data['hangTime']*10).astype(int)
tracking_data_ground = tracking_data[(tracking_data.ground_frame == tracking_data.frameId) & (tracking_data.displayName == 'football')]
tracking_data = pd.merge(tracking_data,tracking_data_ground[['playId','gameId','x','y']] , on=['gameId','playId'],suffixes=['','_ground'])
tracking_data['frame_diff'] = tracking_data['frameId'] - tracking_data['ground_frame']

In [ ]:
tracking_data['first_returner_id'] =  tracking_data.returnerId.str.split(';',n=0).str[0].astype(float)
desinated_returner_list = tracking_data[tracking_data.event == 'kickoff'].groupby(['gameId','playId'])[['x']].idxmax().reset_index()
tracking_data_returner = tracking_data[(tracking_data.event == 'kickoff') & (tracking_data.index.isin(desinated_returner_list['x']))]
tracking_data = pd.merge(tracking_data,tracking_data_returner[['gameId','playId','x','y','nflId']],on=['gameId','playId'],suffixes=['','_returner'])

In [ ]:
tracking_data_kickoff = tracking_data[(tracking_data.event == 'kickoff')&(tracking_data.displayName == 'football')]

In [ ]:
temp_df = tracking_data_kickoff.groupby(['gameId','playId']).first().reset_index()
temp_df = temp_df[~pd.isna(temp_df['kickReturnYardage'])]
X_return_yard  = temp_df[['x_ground','y_ground','x_returner','y_returner']]
y_return_yard = temp_df['kickReturnYardage'].values - temp_df['penaltyYards'].fillna(0).values
X_train, X_test, y_train, y_test = train_test_split(X_return_yard, y_return_yard, test_size=0.2, random_state=42)

In [ ]:
mlp_return_yard = MLPRegressor(random_state=11)
mlp_return_yard.fit(np.array(X_train),np.array(y_train).reshape(-1, 1))
y_pred = mlp_return_yard.predict(np.array(X_test))
print(mean_squared_error(y_test,y_pred))

In [ ]:
temp_df = tracking_data_kickoff.groupby(['gameId','playId']).first().reset_index()

X_oob  = temp_df[['x_ground','y_ground','x_returner','y_returner']]
y_oob = temp_df['specialTeamsResult'] == 'Out of Bounds'
X_train, X_test, y_train, y_test = train_test_split(X_oob, y_oob, test_size=0.1, random_state=42)

In [ ]:
mlp_oob = MLPClassifier(random_state=11)
mlp_oob.fit(np.array(X_train),np.array(y_train).flatten())
y_pred = mlp_oob.predict(np.array(X_test))
print(accuracy_score(y_test,y_pred))

In [ ]:
tnc_oob = TabNetClassifier()
tnc_oob.fit(np.array(X_train),np.array(y_train).flatten(),eval_set=[(np.array(X_test), np.array(y_test))])
y_pred = tnc_oob.predict(np.array(X_test))
print(accuracy_score(y_test,y_pred))

In [ ]:
temp_df = tracking_data_kickoff.groupby(['gameId','playId']).first().reset_index()

X_touchback  = temp_df[['x_ground','y_ground','x_returner','y_returner']]
y_touchback = temp_df['specialTeamsResult'] == 'Touchback'
X_train, X_test, y_train, y_test = train_test_split(X_touchback, y_touchback, test_size=0.2, random_state=42)

In [ ]:
mlp_touchback = MLPClassifier(hidden_layer_sizes=(30,))
mlp_touchback.fit(np.array(X_train),np.array(y_train).flatten())
y_pred = mlp_touchback.predict(np.array(X_test))
print(accuracy_score(y_test,y_pred))

In [ ]:
np.array(y_train).flatten()

In [ ]:
tnc_touchback =  TabNetClassifier()
tnc_touchback.fit(np.array(X_train),np.array(y_train).flatten())
y_pred = tnc_touchback.predict(np.array(X_test))
print(accuracy_score(y_test,y_pred))

In [ ]:
mlp_touchback.predict_proba([[100,25,90,25]])

In [ ]:
tracking_data_kickoff[['x_returner','y_returner']].mean()

In [ ]:
nx, ny = (41, 54)
x = np.linspace(80, 120, nx)
y = np.linspace(0, 53, ny)
xv, yv = np.meshgrid(x, y, sparse = True)

In [ ]:
inference_array = []
coordinate_array = []
for i in xv[0]:
    for j in yv.T[0]:

        inference_array.append([i,j,95,25])
        coordinate_array.append([i,j])
inference_array = np.array(inference_array)
coordinate_array  = np.array(coordinate_array)

In [ ]:
plt.figure(figsize=(12,7),facecolor='white')
c = plt.pcolormesh(xv, yv,mlp_return_yard.predict(np.array(inference_array)).reshape(nx,ny).T)
cbar = plt.colorbar(c)
cbar.set_label('Predicted Return Yardage')
plt.title('Predicted Return Yardage')
plt.vlines(x=110,ymin=0,ymax=53,color='black',linestyle='--')

In [ ]:
return_value = mlp_return_yard.predict(np.array(inference_array)).reshape(nx,ny).T

net_yard_value = np.zeros((54,41))
for i in range(54):
    for j in range(41):

        net_yard_value[i,j] = return_value[i,j] + (41-j)-10

In [ ]:
plt.figure(figsize=(12,7),facecolor='white')
c = plt.pcolormesh(xv, yv,net_yard_value)
cbar = plt.colorbar(c)
cbar.set_label('Predicted Return Yardage')
plt.title('Predicted Return Yardage')


In [ ]:
plt.figure(figsize=(12,7),facecolor='white')
c = plt.pcolormesh(xv, yv,(1-tnc_touchback.predict_proba(np.array(inference_array))[:,1].reshape(nx,ny).T)*net_yard_value+tnc_touchback.predict_proba(np.array(inference_array))[:,1].reshape(nx,ny).T*25)
cbar = plt.colorbar(c)
cbar.set_label('Predicted Starting Yard by kickoff location')
plt.title('Predicted Starting Yard')
plt.vlines(x=110,ymin=0,ymax=53,color='black',linestyle='--')

In [ ]:
from matplotlib import colors
divnorm=colors.TwoSlopeNorm(vmin=-5., vcenter=0., vmax=10)

In [ ]:
plt.figure(figsize=(12,7),facecolor='white')
c = plt.pcolormesh(xv, yv,np.max((net_yard_value,tnc_touchback.predict_proba(np.array(inference_array))[:,1].reshape(nx,ny).T*25),axis=0))
cbar = plt.colorbar(c)
cbar.set_label('Predicted Starting Yard')
plt.title('Predicted Starting Yard')
plt.vlines(x=110,ymin=0,ymax=53,color='black',linestyle='--')

In [ ]:
plt.figure(figsize=(12,7),facecolor='white')
c = plt.pcolormesh(xv, yv,net_yard_value-mlp_touchback.predict_proba(np.array(inference_array))[:,1].reshape(nx,ny).T*25,cmap=plt.get_cmap('RdBu'),norm=divnorm)
plt.xlim([90,120])
cbar = plt.colorbar(c)
cbar.set_label('Yard difference between Touchback and Return')
plt.title('Yard difference between Touchback and Return')
plt.vlines(x=110,ymin=0,ymax=53,color='black',linestyle='--')

In [ ]:
# plt.figure(figsize=(12,7),facecolor='white')
# c = plt.pcolormesh(xv, yv,mlp_touchback.predict_proba(np.array(inference_array))[:,1].reshape(nx,ny).T)
# cbar = plt.colorbar(c)
# cbar.set_label('Touchback Probability')
# plt.title('Touchback Probability')

In [ ]:
plt.figure(figsize=(12,7),facecolor='white')
c = plt.pcolormesh(xv, yv,tnc_touchback.predict_proba(np.array(inference_array))[:,1].reshape(nx,ny).T)
cbar = plt.colorbar(c)
cbar.set_label('Touchback Probability')
plt.title('Predicted Touchback Probability')
plt.vlines(x=110,ymin=0,ymax=53,color='black',linestyle='--')

In [ ]:
plt.figure(figsize=(12,7),facecolor='white')
c = plt.pcolormesh(xv, yv,mlp_oob.predict_proba(np.array(inference_array))[:,1].reshape(nx,ny).T)
cbar = plt.colorbar(c)
cbar.set_label('Out of Bounds Probability')
plt.title('Out of Bounds Probability')
plt.vlines(x=110,ymin=0,ymax=53,color='black',linestyle='--')

In [ ]:
onside_play = kickoff_play[kickoff_play.kickType.isin(['O','S'])]

In [ ]:
onside_play

In [ ]:
tracking_data_onside = []

for year in range(2018,2021):
    data = pd.read_csv('../input/nfl-big-data-bowl-2022/tracking'+str(year) + '.csv')
    data = pd.merge(data,kickoff_play[['gameId','playId']])
#     data = data[data.team == 'football']
    tracking_data_onside.append(data)
tracking_data_onside = pd.concat(tracking_data_onside).reset_index(drop=True)

In [ ]:
tracking_data_onside.loc[tracking_data_onside.playDirection == 'left','x'] = 120-tracking_data_onside['x']
tracking_data_onside.loc[tracking_data_onside.playDirection == 'left','y'] = 53.33-tracking_data_onside['y']

tracking_data_onside['x_dir'] = -np.sin((np.pi / 180) * tracking_data_onside.dir)
tracking_data_onside['y_dir'] = -np.cos((np.pi / 180) * tracking_data_onside.dir)
tracking_data_onside['x_s'] = tracking_data_onside.x_dir * tracking_data_onside.s
tracking_data_onside['y_s'] = tracking_data_onside.y_dir * tracking_data_onside.s

In [ ]:
onside_frame = tracking_data_onside[(tracking_data_onside.event.isin(['onside_kick','autoevent_kickoff','kickoff','kickoff_play','drop_kick','free_kick']))&(tracking_data_onside.displayName == 'football')].groupby(['gameId','playId'])[['frameId','x','y']].first().reset_index()
onside_frame = pd.merge(onside_frame, onside_play[['gameId','playId','hangTime','kickContactType','kickType','kickReturnYardage','playDescription','specialTeamsResult','kickerId','returnerId','penaltyYards']])


In [ ]:
tracking_data_onside= pd.merge(tracking_data_onside,onside_frame[['playId','gameId','frameId','hangTime','x','y','kickContactType','kickType','kickReturnYardage','playDescription','specialTeamsResult','kickerId','returnerId','penaltyYards']],on=['playId','gameId'],suffixes=['','_kickoff'])

In [ ]:
tracking_data_onside['frame_diff'] = tracking_data_onside.frameId - tracking_data_onside.frameId_kickoff

In [ ]:
tracking_data_onside_five = tracking_data_onside[(tracking_data_onside.frame_diff == 5) & (tracking_data_onside.displayName == 'football') & (tracking_data_onside.kickType.isin(['O','S']))]

In [ ]:
tracking_data_onside_five['x_diff'] = tracking_data_onside_five.x - tracking_data_onside_five.x_kickoff
tracking_data_onside_five['y_diff'] = tracking_data_onside_five.y - tracking_data_onside_five.y_kickoff

In [ ]:
tracking_data_onside_five.loc[tracking_data_onside_five['y_diff'] < 0,'y_diff'] = -tracking_data_onside_five['y_diff']

In [ ]:
temp_df = tracking_data_onside_five.groupby(['gameId','playId']).first().reset_index()
X_recover = temp_df[['x_diff','y_diff']]
y_recover = temp_df.specialTeamsResult == 'Kickoff Team Recovery'

In [ ]:
mlp_recover = MLPClassifier()
mlp_recover.fit(X_recover,y_recover)


In [ ]:
nx, ny = (11, 11)
x = np.linspace(0, 10, nx)
y = np.linspace(0, 10, ny)
xv, yv = np.meshgrid(x, y, sparse = True)

In [ ]:
inference_array = []
coordinate_array = []
for i in xv[0]:
    for j in yv.T[0]:

        inference_array.append([i,j])
        coordinate_array.append([i,j])
inference_array = np.array(inference_array)
coordinate_array  = np.array(coordinate_array)

In [ ]:
plt.figure(figsize=(6,6),facecolor='white')
c = plt.pcolormesh(xv, yv,mlp_recover.predict_proba(np.array(inference_array))[:,1].reshape(nx,ny).T)
cbar = plt.colorbar(c)
cbar.set_label('Recover Probability')
plt.title('Recover Probability')
plt.xlabel('x (to endzone ->)')
plt.ylabel('y (to sideline ->)')